In [2]:
import json, time
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup

from requests import get
import requests

from selenium import webdriver
import time

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [4]:
df.head(60)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
5,A27UF1MSF3DB2,0000031887,"C-Lo ""Cynthia""","[0, 0]",I received this today and I'm not a fan of it ...,4.0,It's ok,1396224000,"03 31, 2014"
6,A16GFPNVF4Y816,0000031887,design maven,"[0, 0]",Bought this as a backup to the regular ballet ...,5.0,Great for dress-up and for ballet practice,1399075200,"05 3, 2014"
7,A2M2APVYIB2U6K,0000031887,Jamie P.,"[0, 0]",Great tutu for a great price. It isn't a &#34;...,5.0,Great value,1356220800,"12 23, 2012"
8,A1NJ71X3YPQNQ9,0000031887,JBerger,"[0, 0]","My daughter liked this, and it with her costum...",4.0,Good,1384041600,"11 10, 2013"
9,A3EERSWHAI6SO,0000031887,"Jeffrey Hollingshead ""Jillian hollingshead""","[7, 8]",For what I paid for two tutus is unbeatable an...,5.0,WOW !! ..is all I have to say!,1349568000,"10 7, 2012"


In [5]:
#Check size of dataset 
df.shape

(278677, 9)

In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
df[df['asin'] == 'B00971NLBS']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
224886,A1J4GHI1VRLCX9,B00971NLBS,Alya,"[1, 1]",I will be purchasing more of this brand of shi...,5.0,Classic,1374624000,"07 24, 2013"
224887,A21UXFFQGQS4YJ,B00971NLBS,"Anne Harris ""Lady H""","[0, 0]",I found it to be a little roomy but it will be...,5.0,nice dress shirt,1393545600,"02 28, 2014"
224888,A22Z5LSEIPN8DU,B00971NLBS,Ashleyy,"[0, 0]","I ordered this shirt in a light blue, size two...",2.0,"Wrong color, wrong size.",1372291200,"06 27, 2013"
224889,A3SP43WB89B2UE,B00971NLBS,"cecilafayette ""ceci""","[0, 0]","This shirt fits well, looks nice, and is a gre...",4.0,Great Little Basic Shirt,1388102400,"12 27, 2013"
224890,A1Q3MDPIS017JE,B00971NLBS,Kay Z,"[0, 0]","The material is a little thin, but that was ex...",4.0,"Good fit, little thin",1400284800,"05 17, 2014"
224891,A1BCJXOINBPB38,B00971NLBS,"lascrucesgirl ""Growing boy's momma""","[3, 4]",I did not use the tie for my son. It was dress...,5.0,Nicely made and looks sharp.,1360627200,"02 12, 2013"
224892,A1AFORE8L2LGGW,B00971NLBS,mandamommyof3,"[5, 5]",This is adorable! I would buy more if the tie...,4.0,Tie is too short,1356652800,"12 28, 2012"
224893,A22E4B018G6NX7,B00971NLBS,RDeering,"[0, 0]",Nice little shirt I couldn't find elsewhere th...,5.0,can't complain,1388966400,"01 6, 2014"


In [7]:
#Check how many unique items are in the data 
len(df['asin'].unique())

23033

In [6]:
#Created a non-repeating list of product ID #s with the intent of looping through them in the url to scrape. 
asin_set = set(df['asin'])
asin_set = list(asin_set)

In [9]:
asin_set

['0000031887',
 '0123456479',
 '1608299953',
 '1617160377',
 'B00001W0KA',
 'B00001WRHJ',
 'B00004SR8W',
 'B00004SR8Z',
 'B00004SR9P',
 'B00004U1J2',
 'B000051SEN',
 'B000051SEP',
 'B00005JHKE',
 'B00005JSBK',
 'B00005KJXN',
 'B00005TQI7',
 'B0000643Q8',
 'B000067R84',
 'B00006I551',
 'B00006XXGO',
 'B0000722HK',
 'B000072UMJ',
 'B000074RL3',
 'B000078CYM',
 'B00007AS82',
 'B00007FFL9',
 'B00007GD8X',
 'B00007GD9I',
 'B00007GD9W',
 'B00007GDAL',
 'B00007GDD3',
 'B00007GDG5',
 'B00007IVVR',
 'B000086211',
 'B0000862FI',
 'B0000864CZ',
 'B0000865II',
 'B0000866JI',
 'B000086778',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B0000868O9',
 'B000086910',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B0000891IO',
 'B0000891K0',
 'B00008AALU',
 'B00008ECKG',
 'B00008I8YM',
 'B00008ID0B',
 'B00008ID0L',
 'B00008ID1O',
 'B00008ID39',
 'B00008IEUW',
 'B00008IOHR',
 'B00008IPCG',
 'B00008KH9C',
 'B00008KI3U',
 'B00008LUS5',
 'B00009ESZI',
 'B00009OXE8',
 'B00009QB5V',
 'B00009QMR3',
 'B00009R8

In [15]:
#Check how many unique users are in the data. 
len(df['reviewerID'].unique())

39387

EDA

In [16]:
#Check columns types 
df.dtypes

reviewerID         object
asin               object
reviewerName       object
helpful            object
reviewText         object
overall           float64
summary            object
unixReviewTime      int64
reviewTime         object
dtype: object

In [17]:
#Convert universal time to date
df['clean_time'] = pd.to_datetime(df['unixReviewTime'], unit='s')
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,clean_time
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011",2011-02-12
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013",2013-01-19
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013",2013-01-04
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014",2014-04-27
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014",2014-03-15


In [18]:
#Drop unuseful date and time columns 
df.drop(columns=['reviewTime', 'unixReviewTime'], inplace=True)

In [19]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'clean_time'],
      dtype='object')

In [20]:
df['reviewText'].head()

0    This is a great tutu and at a really great pri...
1    I bought this for my 4 yr old daughter for dan...
2    What can I say... my daughters have it in oran...
3    We bought several tutus at once, and they are ...
4    Thank you Halo Heaven great product for Little...
Name: reviewText, dtype: object

Scrape additional data using Selenium

In [25]:
driver = webdriver.Chrome(executable_path="../chromedriver/macos/chromedriver") 

In [18]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [43]:
t1 = time.time()
print(t1)
list_of_dicts = []
for i in asin_set[21180:]:
    print(f'https://www.amazon.com/gp/product/{i}')
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()

        product_info['description'] = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-none'}).text.strip()
        
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).text.strip()
        
        product_info['size'] = soup.find('div', {'id': 'variation_size_name'})
        
        product_info['details'] = soup.find_all('td', {'class': 'a-span7 a-size-base'})
        
        if soup.find('div', {'id': 'variation_color_name'}):
            product_info['color'] = soup.find('div', {'id': 'variation_color_name'})
            list_of_dicts.append(product_info)
        
        elif soup.find_all('span', {'class': 'selection'}):
            product_info['color'] = soup.find_all('span', {'class': 'selection'})
            list_of_dicts.append(product_info)
            
        elif soup.find_all('div', {'class': 'a-row a-spacing-micro'}):
            product_info['color'] = soup.find_all('div', {'class': 'a-row a-spacing-micro'})
            list_of_dicts.append(product_info)
        else:
            product_info['color'] = 'unavailable'
            list_of_dicts.append(product_info)
    
    except:
        product_info['asin'] = i
        
        product_info['name'] = 'unknown product name'

        product_info['description'] = 'unknown details'
        
        product_info['category'] = 'unknown category'
        
        product_info['color'] = 'unknown color' 
        list_of_dicts.append(product_info)
        print(len(list_of_dicts))
    time.sleep(2)
    
print(time.time() - t1)

1565556453.280046
https://www.amazon.com/gp/product/B00CWU9H9G
https://www.amazon.com/gp/product/B00CWUC9TQ
https://www.amazon.com/gp/product/B00CWVE9KM
https://www.amazon.com/gp/product/B00CWVUVMC
https://www.amazon.com/gp/product/B00CWW1Q1Q
https://www.amazon.com/gp/product/B00CWZACPE
https://www.amazon.com/gp/product/B00CX4WYYQ
https://www.amazon.com/gp/product/B00CX9AG7S
8
https://www.amazon.com/gp/product/B00CX9F3M6
https://www.amazon.com/gp/product/B00CX9SMWO
https://www.amazon.com/gp/product/B00CXGH4R6
https://www.amazon.com/gp/product/B00CXGY0FU
https://www.amazon.com/gp/product/B00CXJNXZK
https://www.amazon.com/gp/product/B00CXK86EM
https://www.amazon.com/gp/product/B00CXKPXCA
https://www.amazon.com/gp/product/B00CXKQ12G
16
https://www.amazon.com/gp/product/B00CXKQC98
https://www.amazon.com/gp/product/B00CXKQRAC
https://www.amazon.com/gp/product/B00CXLBA2G
19
https://www.amazon.com/gp/product/B00CXMN0GY
20
https://www.amazon.com/gp/product/B00CXMP6YS
https://www.amazon.com/gp/

https://www.amazon.com/gp/product/B00D8WL1ZU
https://www.amazon.com/gp/product/B00D8WLBPU
https://www.amazon.com/gp/product/B00D8XPJV6
https://www.amazon.com/gp/product/B00D8XPYOS
182
https://www.amazon.com/gp/product/B00D91TWJW
https://www.amazon.com/gp/product/B00D983OFS
https://www.amazon.com/gp/product/B00D9DAGXG
185
https://www.amazon.com/gp/product/B00D9OATGO
https://www.amazon.com/gp/product/B00DAGEL60
https://www.amazon.com/gp/product/B00DAGKQLY
https://www.amazon.com/gp/product/B00DAGUJQ6
189
https://www.amazon.com/gp/product/B00DAN7H2I
https://www.amazon.com/gp/product/B00DAPTW9C
https://www.amazon.com/gp/product/B00DAQMOO6
192
https://www.amazon.com/gp/product/B00DB24EY2
https://www.amazon.com/gp/product/B00DB2BP44
https://www.amazon.com/gp/product/B00DB8DJLA
195
https://www.amazon.com/gp/product/B00DB9TZ3U
196
https://www.amazon.com/gp/product/B00DBBG1QM
https://www.amazon.com/gp/product/B00DBC1FGM
https://www.amazon.com/gp/product/B00DBC7PPM
https://www.amazon.com/gp/produ

https://www.amazon.com/gp/product/B00DMH1DJA
356
https://www.amazon.com/gp/product/B00DMH6R46
357
https://www.amazon.com/gp/product/B00DMQ5I9W
358
https://www.amazon.com/gp/product/B00DMS7V5O
https://www.amazon.com/gp/product/B00DMSQ60A
360
https://www.amazon.com/gp/product/B00DMWQJ58
https://www.amazon.com/gp/product/B00DMWQJJY
362
https://www.amazon.com/gp/product/B00DMWQK0W
https://www.amazon.com/gp/product/B00DMWQKIY
364
https://www.amazon.com/gp/product/B00DMWQKWA
365
https://www.amazon.com/gp/product/B00DMWQL2E
366
https://www.amazon.com/gp/product/B00DMWQLB0
367
https://www.amazon.com/gp/product/B00DMWQLL0
368
https://www.amazon.com/gp/product/B00DMWQLZG
369
https://www.amazon.com/gp/product/B00DMWQM28
370
https://www.amazon.com/gp/product/B00DMWQMM8
371
https://www.amazon.com/gp/product/B00DMWQN40
https://www.amazon.com/gp/product/B00DMWQNI6
373
https://www.amazon.com/gp/product/B00DMWQO58
https://www.amazon.com/gp/product/B00DMWQOGC
https://www.amazon.com/gp/product/B00DMWQOYY

https://www.amazon.com/gp/product/B00DVIF7BK
https://www.amazon.com/gp/product/B00DVIF8DW
https://www.amazon.com/gp/product/B00DVK75AO
https://www.amazon.com/gp/product/B00DVKQX6G
https://www.amazon.com/gp/product/B00DVLBQY4
536
https://www.amazon.com/gp/product/B00DVPTM8M
537
https://www.amazon.com/gp/product/B00DVQUTC4
https://www.amazon.com/gp/product/B00DVWYL7W
https://www.amazon.com/gp/product/B00DVXYPRM
540
https://www.amazon.com/gp/product/B00DVY6H90
541
https://www.amazon.com/gp/product/B00DVZE1SS
542
https://www.amazon.com/gp/product/B00DW195YG
543
https://www.amazon.com/gp/product/B00DW3AUG6
544
https://www.amazon.com/gp/product/B00DW3AXSG
https://www.amazon.com/gp/product/B00DW44MBY
546
https://www.amazon.com/gp/product/B00DW44ORG
547
https://www.amazon.com/gp/product/B00DW4J110
https://www.amazon.com/gp/product/B00DW4NBW0
https://www.amazon.com/gp/product/B00DW5OEGQ
https://www.amazon.com/gp/product/B00DW6YBTA
https://www.amazon.com/gp/product/B00DW7I0KK
552
https://www.ama

703
https://www.amazon.com/gp/product/B00E4WLQ4Y
704
https://www.amazon.com/gp/product/B00E4WR5JY
705
https://www.amazon.com/gp/product/B00E4WS96W
706
https://www.amazon.com/gp/product/B00E4X9LT0
707
https://www.amazon.com/gp/product/B00E4XD010
708
https://www.amazon.com/gp/product/B00E4YJPCC
https://www.amazon.com/gp/product/B00E5GH5BM
710
https://www.amazon.com/gp/product/B00E5HSMZ4
https://www.amazon.com/gp/product/B00E5J1QQE
712
https://www.amazon.com/gp/product/B00E5KDXQO
713
https://www.amazon.com/gp/product/B00E5LJEQ6
714
https://www.amazon.com/gp/product/B00E5LPPP0
715
https://www.amazon.com/gp/product/B00E5LQWZM
716
https://www.amazon.com/gp/product/B00E5NS53C
717
https://www.amazon.com/gp/product/B00E5OQT5M
718
https://www.amazon.com/gp/product/B00E5PAP6U
719
https://www.amazon.com/gp/product/B00E5PBHSK
https://www.amazon.com/gp/product/B00E5PCVWG
721
https://www.amazon.com/gp/product/B00E5TULBA
722
https://www.amazon.com/gp/product/B00E5UHLBC
https://www.amazon.com/gp/produc

...

1040
https://www.amazon.com/gp/product/B00EUXHMVI
1041
https://www.amazon.com/gp/product/B00EV0DM0A
1042
https://www.amazon.com/gp/product/B00EVB8AHY
https://www.amazon.com/gp/product/B00EVDMBGI
1044
https://www.amazon.com/gp/product/B00EVDNH0M
https://www.amazon.com/gp/product/B00EVDVRBI
1046
https://www.amazon.com/gp/product/B00EVJB0RS
1047
https://www.amazon.com/gp/product/B00EVOHCGQ
1048
https://www.amazon.com/gp/product/B00EVONLSE
1049
https://www.amazon.com/gp/product/B00EVOO9JY
1050
https://www.amazon.com/gp/product/B00EVP2Q5W
1051
https://www.amazon.com/gp/product/B00EVPMBMK
1052
https://www.amazon.com/gp/product/B00EVPPG5Y
1053
https://www.amazon.com/gp/product/B00EVTLHG2
https://www.amazon.com/gp/product/B00EVVX5AQ
1055
https://www.amazon.com/gp/product/B00EVZOENY
https://www.amazon.com/gp/product/B00EW1MLJG
1057
https://www.amazon.com/gp/product/B00EW378YM
1058
https://www.amazon.com/gp/product/B00EWASFQA
1059
https://www.amazon.com/gp/product/B00EWBOHIE
1060
https://www.ama

1207
https://www.amazon.com/gp/product/B00F9GIPAC
1208
https://www.amazon.com/gp/product/B00F9IQ2CS
https://www.amazon.com/gp/product/B00F9K2PZE
1210
https://www.amazon.com/gp/product/B00F9NOYW8
1211
https://www.amazon.com/gp/product/B00F9QAIZM
1212
https://www.amazon.com/gp/product/B00F9QAN3Y
1213
https://www.amazon.com/gp/product/B00F9R8GZA
1214
https://www.amazon.com/gp/product/B00F9RJLTU
1215
https://www.amazon.com/gp/product/B00F9RNVM8
1216
https://www.amazon.com/gp/product/B00F9RZ0W2
https://www.amazon.com/gp/product/B00F9T4XD2
1218
https://www.amazon.com/gp/product/B00FA66SIW
1219
https://www.amazon.com/gp/product/B00FAAH57Q
1220
https://www.amazon.com/gp/product/B00FAAJ5YW
1221
https://www.amazon.com/gp/product/B00FABOFS2
https://www.amazon.com/gp/product/B00FAKKMY4
1223
https://www.amazon.com/gp/product/B00FAKKSAW
1224
https://www.amazon.com/gp/product/B00FALPR70
https://www.amazon.com/gp/product/B00FALPUHW
1226
https://www.amazon.com/gp/product/B00FALPUOK
1227
https://www.ama

1373
https://www.amazon.com/gp/product/B00FSTHA6K
1374
https://www.amazon.com/gp/product/B00FT71IMS
1375
https://www.amazon.com/gp/product/B00FUWST4M
1376
https://www.amazon.com/gp/product/B00FUWST60
1377
https://www.amazon.com/gp/product/B00FVZG3DM
1378
https://www.amazon.com/gp/product/B00FW1DTZA
1379
https://www.amazon.com/gp/product/B00FW3BLW6
1380
https://www.amazon.com/gp/product/B00FW6DGEO
1381
https://www.amazon.com/gp/product/B00FW70YUM
1382
https://www.amazon.com/gp/product/B00FW7UO70
1383
https://www.amazon.com/gp/product/B00FW883V8
1384
https://www.amazon.com/gp/product/B00FWDHKU8
1385
https://www.amazon.com/gp/product/B00FWOYA5U
1386
https://www.amazon.com/gp/product/B00FX4R2RM
1387
https://www.amazon.com/gp/product/B00FX6M5CM
1388
https://www.amazon.com/gp/product/B00FX8YBI6
1389
https://www.amazon.com/gp/product/B00FXE3154
1390
https://www.amazon.com/gp/product/B00FXHTWPO
1391
https://www.amazon.com/gp/product/B00FXLK4ZC
1392
https://www.amazon.com/gp/product/B00FXNNMRC


https://www.amazon.com/gp/product/B00GOKJGMI
1540
https://www.amazon.com/gp/product/B00GOMAM9W
1541
https://www.amazon.com/gp/product/B00GOMDIMU
1542
https://www.amazon.com/gp/product/B00GP28VMG
1543
https://www.amazon.com/gp/product/B00GP2OHEM
1544
https://www.amazon.com/gp/product/B00GPIDWWY
1545
https://www.amazon.com/gp/product/B00GR5G538
1546
https://www.amazon.com/gp/product/B00GRIZZB8
1547
https://www.amazon.com/gp/product/B00GRPZRT6
1548
https://www.amazon.com/gp/product/B00GRPZXFO
1549
https://www.amazon.com/gp/product/B00GRTNLLI
1550
https://www.amazon.com/gp/product/B00GS24ZX2
1551
https://www.amazon.com/gp/product/B00GS5UFU6
1552
https://www.amazon.com/gp/product/B00GS6DZM0
https://www.amazon.com/gp/product/B00GS7XKHO
1554
https://www.amazon.com/gp/product/B00GSFRFY0
1555
https://www.amazon.com/gp/product/B00GSFRHSY
1556
https://www.amazon.com/gp/product/B00GSK838C
1557
https://www.amazon.com/gp/product/B00GSNHLXM
1558
https://www.amazon.com/gp/product/B00GSNXOVU
1559
https

1705
https://www.amazon.com/gp/product/B00I0XXR5E
1706
https://www.amazon.com/gp/product/B00I16VXD8
1707
https://www.amazon.com/gp/product/B00I19G95W
1708
https://www.amazon.com/gp/product/B00I19J12A
1709
https://www.amazon.com/gp/product/B00I19L44I
1710
https://www.amazon.com/gp/product/B00I19LQ5K
1711
https://www.amazon.com/gp/product/B00I1DBKWU
1712
https://www.amazon.com/gp/product/B00I2Q6NE6
1713
https://www.amazon.com/gp/product/B00I2T5OXE
1714
https://www.amazon.com/gp/product/B00I2ZHTHM
1715
https://www.amazon.com/gp/product/B00I3EMKKI
https://www.amazon.com/gp/product/B00I3NPRWW
1717
https://www.amazon.com/gp/product/B00I3O088Y
1718
https://www.amazon.com/gp/product/B00I3Q3NIO
1719
https://www.amazon.com/gp/product/B00I4BMNQQ
1720
https://www.amazon.com/gp/product/B00I4Y9NMU
1721
https://www.amazon.com/gp/product/B00I5M8Y76
1722
https://www.amazon.com/gp/product/B00I5SGUZ8
1723
https://www.amazon.com/gp/product/B00I5TGDZ4
1724
https://www.amazon.com/gp/product/B00I67KRU2
1725


In [44]:
#Check the length of the scrape 
len(list_of_dicts)

1853

In [45]:
#Check the know number of "unknown" rows to make sure it's low 
pd.DataFrame(list_of_dicts)['category'].value_counts()

unknown category                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        1314
Clothing, Shoes & Jewelry\n            \n\n\n             ›\n         \n\n\n                Women\n            \n\n\n             ›\n         \n\n\n                Shoes\n            \n\n\n             ›\n         \n\n\n                 Flats                                                                                                                                                                                         

In [14]:
#product8000 = pd.DataFrame(list_of_dicts)

In [16]:
product8000.to_csv('product8000.csv')

In [22]:
#product8534 = pd.DataFrame(list_of_dicts)

In [53]:
product8534.shape

(534, 7)

In [54]:
product8534.to_csv('data/product8534')

In [29]:
#product14000 = pd.DataFrame(list_of_dicts)

In [30]:
product14000.to_csv('product14000.csv')

In [35]:
#product20000 = pd.DataFrame(list_of_dicts)

In [36]:
product20000.to_csv('product20000.csv')

In [41]:
#product21180 = pd.DataFrame(list_of_dicts)

In [42]:
product21180.to_csv('product21180.csv')

In [46]:
# product23033 = pd.DataFrame(list_of_dicts)

In [47]:
product23033.to_csv('product23033.csv')

Many of the above scrapes were blocked by an anti-robot page so i'll need to go through and check for unavailable rows and make a new list of asins to rescrape.

In [55]:
# Function to remove html
def remove_html(text):
    try:
        soup = BeautifulSoup(text, 'lxml')
        html_free = soup.get_text(strip=True)
        return html_free
    except:
        return "missing"

In [56]:
# Function to clean the DataFrame
def EDA(df):
    # Remove html and strip off brackets and 'Color:' from color column
    df['color'] = df['color'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '').str.replace('Color:', '')
    
    # Replace escape characters and white space in Category column 
    df['category'] = df['category'].str.replace('\n', '').str.replace('  ', '')
    
    # Split Categroy column on the '›' symbol, up to 6 times and return them in a new df where each split is a new column 
    category = df['category'].str.split("›", n=6, expand=True)
    
    # Rename each category split column and add it onto the original df
    df['department'] = category[0]
    df['demographic'] = category[1]
    df['division'] = category[2]
    df['category'] = category[3]
    df['subcategory'] = category[4]
    df['type'] = category[5]
    df['detail_type'] = category[6]
    
    # Remove special characters from the description column 
    df['description'] = df['description'].str.replace('\t', '').str.replace('\n', '')
    
    # Remove html and strip off brackets from details column
    df['details'] = df['details'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '')
   
    #Remove html and cut off 'Size:' from the size column 
    df['size'] = df['size'].apply(lambda x: remove_html(x)).str.replace('Size:', '')
    
    return df.head()

In [57]:
EDA(product8000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,0000031887,Dance,unavailable,This fits your . Make sure this fitsby ent...,missing,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,missing,Sports & Outdoors,Sports & Fitness,Other Sports,Clothing,Girls,Skirts
1,0123456479,Jewelry Boxes & Organizers,unavailable,This fits your . Make sure this fitsby ent...,missing,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,missing,"Clothing, Shoes & Jewelry","Shoe, Jewelry & Watch Accessories",Jewelry Accessories,Jewelry Boxes,None,None
2,1608299953,None,unavailable,Access for up to 5 family members Download act...,missing,Learn French: Rosetta Stone French - Level 1,missing,Software,Education & Reference,Languages,None,None,None
3,1617160377,None,unavailable,Access for up to 5 family members Download act...,missing,Learn Italian: Rosetta Stone Italian - Level 1,missing,Software,Education & Reference,Languages,None,None,None
4,B00001W0KA,None,missing,Lead Free Child (4-6 & 7-8) Includes: Bodysuit...,missing,Buzz Lightyear Boy's Deluxe Toy Story Costume,missing,Toys & Games,Dress Up & Pretend Play,Costumes,None,None,None


In [59]:
redo_df1 =product8000[product8000['department'] == 'unknown category']

In [60]:
redo_df1.shape

(5736, 13)

In [62]:
redo_list = list(redo_df1['asin'])
redo_list

['B00005TQI7',
 'B0000722HK',
 'B00007GD9I',
 'B0000864CZ',
 'B0000866JI',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B00008ID0B',
 'B00008ID1O',
 'B00008ID39',
 'B00008IOHR',
 'B00008KI3U',
 'B00008LUS5',
 'B00009QB5V',
 'B0000AT3NT',
 'B0000B35D8',
 'B0000TIIRS',
 'B0000X5EJO',
 'B0000X5FAM',
 'B0000YEC8C',
 'B0000YECC8',
 'B0000ZCH4W',
 'B000147UNU',
 'B00014861K',
 'B00018HRU2',
 'B0001IA06U',
 'B0001VJ2V6',
 'B0001VWSFS',
 'B0001WD5RC',
 'B0001XH7X4',
 'B0001YWBG6',
 'B000200JQI',
 'B00020ZUC6',
 'B00023K3YS',
 'B00029BKAI',
 'B0002DG7ZM',
 'B0002EJ4XS',
 'B0002FHFZQ',
 'B0002FHGBO',
 'B0002FHIM6',
 'B0002FHIMQ',
 'B0002FHIVM',
 'B0002FHIWQ',
 'B0002JA0WC',
 'B0002L01YC',
 'B0002LKLPQ',
 'B0002NZ0LO',
 'B0002TVB70',
 'B0002U71R8',
 'B0002VG2XG',
 'B0002XROVS',
 'B0002ZOQZ8',
 'B00031XFL2',
 'B00061RG1O',
 'B00063J8MW',
 'B00063W3GA',
 'B00064UU1Y',
 'B000662EL6',
 'B00066G516',
 'B000673JT6',
 'B00068VGOU',
 'B0006GK8V4',
 'B0006IXC

In [63]:
EDA(product20000)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B006W71WJI,Mules & Clogs,missing,100% Leather Synthetic sole Heel measures appr...,missing,Dansko Women's Valerie Pull-Up Clog,missing,"Clothing, Shoes & Jewelry",Women,Shoes,None,None,None
1,B006W8QR56,Drawstring Bags,missing,IDEAL SIZE -Drawstring backpacks come in one s...,missing,Mato & Hash Drawstring Bulk Bags Cinch Sacks B...,missing,"Clothing, Shoes & Jewelry",Luggage & Travel Gear,Gym Bags,None,None,None
2,B006W8WLTW,Clothing,missing,100% Interlock Cotton Printed Sayings Machine ...,missing,Luvable Friends Baby Sayings Bodysuit – Wild Boy,missing,"Clothing, Shoes & Jewelry",Baby,Baby Boys,Bodysuits,None,None
3,B006W8WMP0,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B006WANIVK,"Tops, Tees & Blouses",missing,90% Cotton | 10% Rayon Made in USA Gentle cycl...,missing,Funfash Plus Size Women Open Angel Sleeve Cold...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Tunics,None,None


In [64]:
product20000.isnull().sum()

asin              0
category       4334
color             0
description       0
details           0
name              0
size              0
department        0
demographic    4272
division       4289
subcategory    4857
type           5692
detail_type    5869
dtype: int64

In [65]:
product20000['department'].value_counts(dropna=False)

unknown category              4272
Clothing, Shoes & Jewelry     1659
Sports & Outdoors               44
Toys & Games                     8
Arts, Crafts & Sewing            5
Baby Products                    3
Health & Household               3
Beauty & Personal Care           2
Tools & Home Improvement         1
Office Products                  1
Electronics                      1
Home & Kitchen                   1
Name: department, dtype: int64

In [67]:
redo_df2 = product20000[product20000['department'] == 'unknown category']

In [72]:
redo_list.extend(list(redo_df2['asin']))

In [73]:
len(redo_list)

10009

In [74]:
EDA(product21180)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00B8ZJ8U4,Body Jewelry,unavailable,THIS PRICE IS FOR TWO BRACELETS! This bracelet...,missing,JOTW 2 Pieces of Goldtone with Clear Iced Out ...,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,None,None,None
1,B00B93TTHM,Dresses,missing,100% Polyester Hand Wash (Recommended Dry Clea...,missing,Doublju Classic Slim Fit Sleeveless Sexy Bodyc...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Club & Night Out,None,None
2,B00B93UDRW,Dresses,missing,65% Cotton / 35% Polyester Machine Washable (R...,missing,Doublju Slim Fit Ribbed Knit Button Down Henle...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Casual,None,None
3,B00B93W5R8,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B00B98MPWS,Athletic,missing,Synthetic Imported Rubber sole 0mm heel-to-toe...,missing,Merrell Men's Ascend Glove Minimal Running Shoe,missing,"Clothing, Shoes & Jewelry",Men,Shoes,Running,Trail Running,None


In [75]:
product21180.isnull().sum()

asin              0
category        702
color             0
description       0
details           0
name              0
size              0
department        0
demographic     687
division        691
subcategory     845
type           1097
detail_type    1146
dtype: int64

In [77]:
product21180['department'].value_counts()

unknown category              687
Clothing, Shoes & Jewelry     478
Sports & Outdoors              11
Electronics                     2
Health & Household              1
Toys & Games                    1
Name: department, dtype: int64

In [78]:
redo_df3 = product21180[product21180['department'] == 'unknown category']

In [79]:
redo_list.extend(list(redo_df3['asin']))
len(redo_list)

10696

In [80]:
EDA(product23033)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00CWU9H9G,Sweaters,missing,100% Rayon Short sleeve shrug flyaway cardigan...,missing,Rayon Short Sleeves Flyaway Cardigan Shawl Col...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Cardigans,None,None
1,B00CWUC9TQ,Yoga,missing,Imported Featuring a fold over waist and flare...,missing,"Plus Size Yoga Long Stretch Pants 92% Cotton, ...",missing,Sports & Outdoors,Sports & Fitness,Exercise & Fitness,Clothing,Women,Pants
2,B00CWVE9KM,"Tops, Tees & Blouses",missing,"Scoop neck short sleeve top, super comfy Plus ...",missing,Hollywood Star Fashion Women's Short Cap Sleev...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,T-Shirts,None,None
3,B00CWVUVMC,Wrist Watches,unavailable,"CLASSIC DESIGN: PU leather band, roman number ...",missing,"Mens Quartz Watch, Roman Numeral Business Casu...",missing,"Clothing, Shoes & Jewelry",Men,Watches,None,None,None
4,B00CWW1Q1Q,"Tops, Tees & Blouses",missing,Chiffon Sheer Long Sleeves Blouse 100% Polyest...,missing,Basic Long Sleeves Sheer Chiffon Blouse with P...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Blouses & Button-Down Shirts,None,None


In [81]:
product23033.isnull().sum()

asin              0
category       1335
color             0
description       0
details           0
name              0
size              0
department        0
demographic    1314
division       1322
subcategory    1474
type           1770
detail_type    1826
dtype: int64

In [82]:
product23033['department'].value_counts(dropna=False)

unknown category              1314
Clothing, Shoes & Jewelry      520
Sports & Outdoors               11
Toys & Games                     3
Arts, Crafts & Sewing            2
Tools & Home Improvement         1
Health & Household               1
Electronics                      1
Name: department, dtype: int64

In [83]:
redo_df4 = product23033[product23033['department'] == 'unknown category']

In [84]:
redo_list.extend(list(redo_df4['asin']))
len(redo_list)

12010